<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/WGAN_With_GRADIENT_PENALTY_(GP)_Pytorch_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>>You are on Google CoLaB with torch version {torch.__version__}")
except Exception as e:
  print(f">>>>{type(e)}: {e}\n>>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
def time_fmt(t: float = 213.981)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h} hrs: {m:>02} min: {s:>05.2f} sec"
print(f">>>>time formating\tplease wait.....\n>>>>time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>>You are on Google CoLaB with torch version 1.8.1+cu101
>>>>time formating	please wait.....
>>>>time elapsed	0 hrs: 03 min: 33.00 sec


In [60]:
#In this notebook we are going to implement WGAN network with gradient penalty (GP) to 
#improve the Wasserstein Loss. The architecture is purely WGAN with additional features
#and Adam optimizer with zero momentum.

In [61]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np
import math, sys,time,random, torchvision

In [62]:
#Setup the seed for reproducability and GPU to deterministic to avoid errors during training:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [63]:
#The architecture of this model is purelly WGAN where the discriminator is a CNN with Convolutional layers
#and the generator is using de-convolution layers to reconstruct fake images/data.

In [64]:
class Discriminator(nn.Module):
  def __init__(self, img_channels, d_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Conv2d(img_channels, d_features, kernel_size = 4, stride = 2, padding = 1),
        nn.LeakyReLU(0.2),
        self.__dblock__(d_features, 2*d_features, 4, 2, 1),
        self.__dblock__(2*d_features, 4*d_features, 4, 2, 1),
        self.__dblock__(4*d_features, 8*d_features, 4, 2, 1),
        nn.Conv2d(8*d_features, 1,kernel_size = 4, stride = 2, padding = 0))
  def __dblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels,
                  out_channels,
                  kernel_size,
                  stride,
                  padding,
                  bias = False),
        nn.InstanceNorm2d(out_channels, affine = True),
        nn.LeakyReLU(0.2))
  
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)

In [65]:
class Generator(nn.Module):
  def __init__(self, z_dim, img_channels, g_features):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        self.__gblock__(z_dim, g_features * 16, 4, 2, 0),
        self.__gblock__(g_features*16, g_features*8, 4, 2, 1),
        self.__gblock__(g_features*8, g_features*4, 4, 2, 1),
        self.__gblock__(g_features*4, g_features*2, 4, 2, 1),
        nn.ConvTranspose2d(2*g_features, img_channels, kernel_size = 4, stride = 2, padding = 1),
        nn.Tanh())
    
  def __gblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels,
                           out_channels,
                           kernel_size,
                           stride,
                           padding,
                           bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
  
  def forward(self, input_tensor):
    return self.generator(input_tensor)

In [66]:
#Testing the discriminator and the generator networks to see if they produce the intended results.

In [67]:
def __test__():
  z_dim = 100
  batch_size = 128
  img_channels = 3
  W, H = 64, 64
  lamda_gp = 10
  noise_img = torch.randn(batch_size, z_dim, 1,1)
  rand_img = torch.randn(batch_size, img_channels, W,H)
  disc = Discriminator(img_channels, 8)
  gen = Generator(z_dim, img_channels, 8)
  disc_out = disc(rand_img)
  gen_out = gen(noise_img)
  return gen_out.shape, disc_out.shape


In [68]:
__test__()

(torch.Size([128, 3, 64, 64]), torch.Size([128, 1, 1, 1]))

In [69]:
#Initialize the parameters of the model to random normal:
def __par__(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, mean = 0.00, std = 0.02)

In [70]:
#Instantiate the model classes and initialize the parameters
z_dim = 100
img_size = 64
lambda_gp = 10
batch_size = 64
g_features = 64
d_features = 64
img_channels = 1
learning_rate = 1e-4
discriminator = Discriminator(img_channels, d_features).to(device = device)
generator = Generator(z_dim, img_channels, g_features).to(device = device)
fixed_noise = torch.randn(batch_size, z_dim, 1,1).to(device = device) #to generate fake images after training the generator
EPOCHS = 10
disc_iter = 5
__par__(discriminator)
__par__(generator)


In [71]:
#Now we define the gradient penalty like as follow
def __gp__(discriminator, real_img, fake_img, device = device):
  '''this method/function will performs interpolation of the two images (real + fake)
  and then linear combination of the Wasserstein Loss will be computed to replace 
  the art of gradient cliping that affect training by lower the slopes.'''
  batch_size, img_channels, W, H = real_img.shape
  e = torch.rand((batch_size, 1, 1, 1)).repeat(1, img_channels, W, H).to(device = device) #generate the epsilon(rand # ~U[0,1])
  ipltn_images = e*real_img + (1 - e)* fake_img #This interpolate images to create a mixed kind of data (both real and fake)
  #Computing the discriminant scores using the new mixed data
  disc_scores = discriminator(ipltn_images)
  #computing the gradient using autograd:
  grads = torch.autograd.grad(
      inputs = ipltn_images,
      outputs = disc_scores,
      grad_outputs = torch.ones_like(disc_scores),
      create_graph = True, retain_graph = True)[0]
  #Reshaping the gradient tensor and apply L2-norm as suggested in the WGAN with Gradient penalty paper
  grads = grads.view(grads.shape[0], -1) #grabing the slopes and flattening to 1d
  grad_norm = grads.norm(2, dim = 1) #compute norm-2
  gp = torch.mean((grad_norm - 1)**2) #computing the gradient penalty value (g-p)
  return gp


In [72]:
#Get the optimizers (ADAM) for both network with no momentum and the tensoboard writers

In [73]:
disc_optimizer = optim.Adam(params = discriminator.parameters(), lr = learning_rate, betas = (0.00, 0.9))
gen_optimizer = optim.Adam(params = generator.parameters(), lr = learning_rate, betas = (0.00, 0.9))
real_writer = SummaryWriter(f"logs/real_images")
fake_writer = SummaryWriter(f"loga=s/fake_images")

In [74]:
#Loading and preprocessing the mnist data from torchvision using the transform method
transforms = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Resize(img_size),
                                 transforms.Normalize(
                                [0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)])])
dfm = datasets.MNIST(root = "mnist_data/", transform = transforms, download = True)
loader = DataLoader(dataset = dfm, shuffle = True, batch_size = batch_size)
x_loader, y_loader = next(iter(loader))
print(f">>>>x_loader_shape: {x_loader.shape}\ty_loader_shape: {y_loader.shape}")

>>>>x_loader_shape: torch.Size([64, 1, 64, 64])	y_loader_shape: torch.Size([64])


In [ ]:
#The training loop:
global_tic = time.time()
step = 0
for epoch in range(EPOCHS):
  tic = time.time()
  print(f"\n>>>>training starts for epoch: {epoch + 1}\tplease wait while the model is training.......\n>>>>training on progress: KEEP YOUR SCREEN ACTIVE.......")
  for idx, (data,_) in enumerate(tqdm(loader)):
    data = data.to(device = device)
    batch_size = data.shape[0]
    noise = torch.randn(batch_size, z_dim, 1, 1).to(device = device)
    fake = generator(noise)
    
    #training the discriminator we maximize the loss [log(D(x)) + log(1-D(G(z)))]:
    for _ in range(disc_iter):
      noise = torch.randn(batch_size, z_dim, 1, 1).to(device = device)
      fake = generator(noise)
      disc_out_real = discriminator(data).reshape(-1)
      disc_out_fake = discriminator(fake).reshape(-1)
      gp = __gp__(discriminator, data, fake, device)
      disc_loss = (-(torch.mean(disc_out_real) - torch.mean(disc_out_fake)) + lambda_gp*gp) #Wasserstein Loss with gradient penalty
      discriminator.zero_grad()#initialize the gradients for the discriminator model to 0
      disc_loss.backward(retain_graph = True) #backward pass for the discriminator
      disc_optimizer.step()#gradient descent using adam optimizer with no mementum

    #training the generator: we maximize [log(D(G(z)))] instead of minimizing log(1 - D(G(z)))
    out_gen = discriminator(fake)
    gen_loss = -(torch.mean(out_gen)) #Normal Wasserstein Loss with no gp
    generator.zero_grad() #initialize the slopes to zeros
    gen_loss.backward() #backward pass for the generator
    gen_optimizer.step()# generator's gradient descent with adam optimizer with zero momrntum
    toc = time.time()

    if idx % 100 == 0:
      print(f"\n>>>> time at the end of epoch: {epoch + 1} for batch {idx} is: {time_fmt(toc - tic)}")
      print(f"\n>>>> generator loss: {gen_loss:.4f} | generator PPL: {math.exp(gen_loss):7.4f}")
      print(f">>>> discriminator loss: {disc_loss:.4f} | discriminator PPL: {math.exp(disc_loss):7.4f}")

    with torch.no_grad():
      fake_img = generator(fixed_noise)
      real_img_grid = torchvision.utils.make_grid(data[:128], normalize = True)
      fake_img_grid = torchvision.utils.make_grid(fake_img[:128], normalize = True)
      real_writer.add_image("real_image",real_img_grid, global_step = step)
      fake_writer.add_image("fake_image", fake_img_grid, global_step = step)
    step+=1
global_toc = time.time()
print(f"\n>>>>total training time for 5 epochs: {time_fmt(global_toc - global_tic)}")












  0%|          | 0/938 [00:00<?, ?it/s]


>>>>training starts for epoch: 1	please wait while the model is training.......
>>>>training on progress: KEEP YOUR SCREEN ACTIVE.......

>>>> time at the end of epoch: 1 for batch 0 is: 0 hrs: 00 min: 01.00 sec

>>>> generator loss: 0.5765 | generator PPL:  1.7799
>>>> discriminator loss: 64.6102 | discriminator PPL: 11477735245519738681165873152.0000








  0%|          | 1/938 [00:01<26:02,  1.67s/it]





  0%|          | 2/938 [00:03<26:13,  1.68s/it]





  0%|          | 3/938 [00:05<26:19,  1.69s/it]





  0%|          | 4/938 [00:06<26:22,  1.69s/it]





  1%|          | 5/938 [00:08<26:22,  1.70s/it]





  1%|          | 6/938 [00:10<26:31,  1.71s/it]





  1%|          | 7/938 [00:11<26:33,  1.71s/it]





  1%|          | 8/938 [00:13<26:40,  1.72s/it]





  1%|          | 9/938 [00:15<26:43,  1.73s/it]





  1%|          | 10/938 [00:17<26:47,  1.73s/it]





  1%|          | 11/938 [00:18<26:52,  1.74s/it]





  1%|▏         | 12/938 [00:20<26:56,  1.75s/it]





  1%|▏         | 13/938 [00:22<26:55,  1.75s/it]





  1%|▏         | 14/938 [00:24<26:53,  1.75s/it]





  2%|▏         | 15/938 [00:25<26:53,  1.75s/it]





  2%|▏         | 16/938 [00:27<26:51,  1.75s/it]





  2%|▏         | 17/938 [00:29<26:54,  1.75s/it]





  2%|▏         | 18/938 [00:31<26:58,  1.76s/it]





  2%|▏         | 19/938 [00:33<


>>>> time at the end of epoch: 1 for batch 100 is: 0 hrs: 02 min: 51.00 sec

>>>> generator loss: 107.9240 | generator PPL: 74264739470520096126691301342876539379240665088.0000
>>>> discriminator loss: -122.5349 | discriminator PPL:  0.0000








 11%|█         | 101/938 [02:51<23:13,  1.67s/it]





 11%|█         | 102/938 [02:53<23:09,  1.66s/it]





 11%|█         | 103/938 [02:55<23:08,  1.66s/it]





 11%|█         | 104/938 [02:56<23:03,  1.66s/it]





 11%|█         | 105/938 [02:58<23:03,  1.66s/it]





 11%|█▏        | 106/938 [02:59<22:59,  1.66s/it]





 11%|█▏        | 107/938 [03:01<22:57,  1.66s/it]





 12%|█▏        | 108/938 [03:03<22:56,  1.66s/it]





 12%|█▏        | 109/938 [03:04<22:52,  1.66s/it]





 12%|█▏        | 110/938 [03:06<22:53,  1.66s/it]





 12%|█▏        | 111/938 [03:08<22:51,  1.66s/it]





 12%|█▏        | 112/938 [03:09<22:51,  1.66s/it]





 12%|█▏        | 113/938 [03:11<22:50,  1.66s/it]





 12%|█▏        | 114/938 [03:13<22:47,  1.66s/it]





 12%|█▏        | 115/938 [03:14<22:45,  1.66s/it]





 12%|█▏        | 116/938 [03:16<22:45,  1.66s/it]





 12%|█▏        | 117/938 [03:18<22:43,  1.66s/it]





 13%|█▎        | 118/938 [03:19<22:40,  1.66s/it]





 13%

...







 85%|████████▌ | 801/938 [21:12<03:37,  1.59s/it]





 86%|████████▌ | 802/938 [21:13<03:35,  1.59s/it]





 86%|████████▌ | 803/938 [21:15<03:34,  1.59s/it]





 86%|████████▌ | 804/938 [21:17<03:32,  1.59s/it]





 86%|████████▌ | 805/938 [21:18<03:30,  1.58s/it]





 86%|████████▌ | 806/938 [21:20<03:29,  1.59s/it]





 86%|████████▌ | 807/938 [21:21<03:27,  1.59s/it]





 86%|████████▌ | 808/938 [21:23<03:26,  1.59s/it]





 86%|████████▌ | 809/938 [21:24<03:24,  1.58s/it]





 86%|████████▋ | 810/938 [21:26<03:22,  1.58s/it]





 86%|████████▋ | 811/938 [21:28<03:21,  1.59s/it]





 87%|████████▋ | 812/938 [21:29<03:19,  1.59s/it]





 87%|████████▋ | 813/938 [21:31<03:18,  1.59s/it]





 87%|████████▋ | 814/938 [21:32<03:17,  1.59s/it]





 87%|████████▋ | 815/938 [21:34<03:15,  1.59s/it]





 87%|████████▋ | 816/938 [21:36<03:14,  1.59s/it]





 87%|████████▋ | 817/938 [21:37<03:12,  1.59s/it]





 87%|████████▋ | 818/938 [21:39<03:10,  1.59s/it]





 87%


>>>> time at the end of epoch: 4 for batch 900 is: 0 hrs: 23 min: 50.00 sec

>>>> generator loss: 57.5690 | generator PPL: 10043441810323134444732416.0000
>>>> discriminator loss: -8.1838 | discriminator PPL:  0.0003








 96%|█████████▌| 901/938 [23:51<00:58,  1.59s/it]





 96%|█████████▌| 902/938 [23:52<00:57,  1.59s/it]





 96%|█████████▋| 903/938 [23:54<00:55,  1.59s/it]





 96%|█████████▋| 904/938 [23:55<00:53,  1.59s/it]





 96%|█████████▋| 905/938 [23:57<00:52,  1.59s/it]





 97%|█████████▋| 906/938 [23:58<00:50,  1.59s/it]





 97%|█████████▋| 907/938 [24:00<00:49,  1.59s/it]





 97%|█████████▋| 908/938 [24:02<00:47,  1.59s/it]





 97%|█████████▋| 909/938 [24:03<00:46,  1.59s/it]





 97%|█████████▋| 910/938 [24:05<00:44,  1.59s/it]





 97%|█████████▋| 911/938 [24:06<00:42,  1.59s/it]





 97%|█████████▋| 912/938 [24:08<00:41,  1.59s/it]





 97%|█████████▋| 913/938 [24:10<00:39,  1.59s/it]





 97%|█████████▋| 914/938 [24:11<00:38,  1.59s/it]





 98%|█████████▊| 915/938 [24:13<00:36,  1.59s/it]





 98%|█████████▊| 916/938 [24:14<00:34,  1.59s/it]





 98%|█████████▊| 917/938 [24:16<00:33,  1.59s/it]





 98%|█████████▊| 918/938 [24:18<00:31,  1.59s/it]





 98%


>>>>training starts for epoch: 5	please wait while the model is training.......
>>>>training on progress: KEEP YOUR SCREEN ACTIVE.......

>>>> time at the end of epoch: 5 for batch 0 is: 0 hrs: 00 min: 01.00 sec

>>>> generator loss: 58.3313 | generator PPL: 21525730066325513130999808.0000
>>>> discriminator loss: -5.7285 | discriminator PPL:  0.0033








  0%|          | 1/938 [00:01<24:05,  1.54s/it]





  0%|          | 2/938 [00:03<24:18,  1.56s/it]





  0%|          | 3/938 [00:04<24:25,  1.57s/it]





  0%|          | 4/938 [00:06<24:27,  1.57s/it]





  1%|          | 5/938 [00:07<24:28,  1.57s/it]





  1%|          | 6/938 [00:09<24:28,  1.58s/it]





  1%|          | 7/938 [00:11<24:29,  1.58s/it]





  1%|          | 8/938 [00:12<24:27,  1.58s/it]





  1%|          | 9/938 [00:14<24:28,  1.58s/it]





  1%|          | 10/938 [00:15<24:32,  1.59s/it]





  1%|          | 11/938 [00:17<24:28,  1.58s/it]





  1%|▏         | 12/938 [00:19<24:25,  1.58s/it]





  1%|▏         | 13/938 [00:20<24:21,  1.58s/it]







In [ ]:
!tensorboard -logdir = logs